# **Demo:** Net2WiderNet on ImageNet with Inception-V2

The following demo shows how to apply Net2WiderNet to Inception-V2 in order to increase the number of output filters in each layer of the Inception blocks. The input image shape is the one of ImageNet, but the network and the Net2WiderNet algorithm can be applied to any other image size.

In [ ]:
# Import libraries
import torch
import numpy as np
import torchinfo

# Import custom modules and packages
from models.inceptionv2 import GoogleNetBN
import params.inceptionv2_imagenet
import net2net.net2net_wider

### 1. Create an Inception-V2 model narrower than the original one

We start by creating an Inception-V2 model, narrower than the standard model: the number of convolution channels at each layer within all Inception modules is reduced by a factor of $\sqrt{0.3}$. The rest of the network remains the same.

In [ ]:
# Create a downsized version of the Inception-V2 network
# (with 10 classes instead of 1000 for demo purposes)
model = GoogleNetBN(nb_classes=10, inception_factor=np.sqrt(0.3))

# Create a random input
x = torch.randn(1,
                params.inceptionv2_imagenet.NB_CHANNELS,
                *params.inceptionv2_imagenet.IMAGE_SHAPE)

# Compute the output of the teacher network
# (forward pass to initialize the Lazy modules)
y_teacher = model(x)

### 2. Recover the standard architecture of Inception-V2 using the Net2WiderNet algorithm

We then apply the Net2WiderNet algorithm to the narrower model in order to recover the standard architecture of Inception-V2. The algorithm is applied to the Inception modules and the fully-connected layer only, since the rest of the network is already standard. The weights and biases of the student model (the wider one) are initialized with those of the teacher model (the narrower one), in such a way that the output of the student model is the same as the output of the teacher model for the same input at initialization.

In [ ]:
# Instantiate a Net2Net object from a (pre-trained) model
net2net = net2net.net2net_wider.Net2Net(teacher_network=model)

# Get the list of widening operations
wider_operations = params.inceptionv2_imagenet.wider_operations

# Add some noise to the copied weights (optional)
sigma = 0.  # Standard deviation of the noise

# Apply the Net2Net widening operations and get the student network
net2net.net2wider(wider_operations, sigma=sigma)
student_model = net2net.get_student_network()

# Compute the output of the student network
y_student = student_model(x)

### 3. Check that the student and teacher models have the same output for the same input

We check that the output of the student model is the same as the output of the teacher model for the same input at initialization. They can be slightly different if some noise has been added to the weights of the student model during the initialization.

In [ ]:
# The outputs should be the same
print("Teacher output: ", y_teacher)
print("Student output: ", y_student, "\n")

### 4. Have a look at the student and teacher architectures

We start by displaying the architecture of the teacher model. We can check that the number of convolution channels at each layer within all Inception modules is reduced by a factor of $\sqrt{0.3}$. The model has $1.886.577$ trainable parameters.

In [ ]:
# Display the architecture of the student network
torchinfo.summary(model, input_size=(1,
                                     params.inceptionv2_imagenet.NB_CHANNELS,
                                     *params.inceptionv2_imagenet.IMAGE_SHAPE))

We then display the architecture of the student model. We can check that the number of convolution channels at each layer within all Inception modules is the same as the standard model. The model has $5.998.362$ trainable parameters. Thus, the number of parameters in the teacher model is about $31.5\%$ of the number of parameters in the student model.

In [ ]:
# Display the architecture of the student network
torchinfo.summary(net2net.student_network, input_size=(1,
                                                       params.inceptionv2_imagenet.NB_CHANNELS,
                                                       *params.inceptionv2_imagenet.IMAGE_SHAPE))